In [1]:
pip install sklearn-crfsuite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
import warnings
warnings.filterwarnings('ignore') 

In [3]:
df = pd.read_csv('/kaggle/input/vner-vlsp-2021/processed_data.csv')

In [4]:
# df = df.sample(frac=0.01, random_state=42)
df.head(10)

,Word,Tag,Sentence #
0,Mong ước,O,Sentence: 1
1,được,O,Sentence: 1
2,đến,O,Sentence: 1
3,trường,O,Sentence: 1
4,của,O,Sentence: 1
5,bé,O,Sentence: 1
6,gái,O,Sentence: 1
7,7,B-QUANTITY-AGE,Sentence: 1
8,tuổi,I-QUANTITY-AGE,Sentence: 1
9,Giờ,B-DATETIME,Sentence: 2


In [5]:
tokenized_sentences = {}
for index, row in df.iterrows():
    sentence_id = row['Sentence #']
    word = row['Word']
    
    # Nếu câu chưa có trong từ điển, khởi tạo danh sách
    if sentence_id not in tokenized_sentences:
        tokenized_sentences[sentence_id] = []
    
    # Thêm từ vào danh sách của câu
    tokenized_sentences[sentence_id].append(word)

 
 # Chuyển đổi từ điển thành danh sách các câu
input_data = list(tokenized_sentences.values())
input_data[0]

['Mong ước', 'được', 'đến', 'trường', 'của', 'bé', 'gái', '7', 'tuổi']

In [6]:
import pickle
from gensim.models import FastText


# Huấn luyện mô hình FastText
model_fasttext = FastText(input_data, vector_size=100, window=3, min_count=1, sg=1)


with open('/kaggle/working/fasttext_model_f.pkl', 'wb') as file:
    pickle.dump(model_fasttext, file)
# Tải lại mô hình từ file word2vec_model.pkl
with open('/kaggle/working/fasttext_model_f.pkl', 'rb') as f:
    loaded_model = pickle.load(f)
    
print("Mô hình FastText đã được huấn luyện và lưu thành công.")

Mô hình FastText đã được huấn luyện và lưu thành công.


In [7]:
import numpy as np

# Hàm để lấy vector cho từng từ
def get_word_vector(word, model):
    if word in model.wv:
        return model.wv[word]
    else:
        return np.zeros(model.vector_size)

# Tạo vector cho từng từ trong DataFrame
df['Vector'] = df['Word'].apply(lambda x: get_word_vector(x, model_fasttext).tolist())
df.head(10)

,Word,Tag,Sentence #,Vector
0,Mong ước,O,Sentence: 1,"[0.16203883290290833, 0.001150104682892561, 0...."
1,được,O,Sentence: 1,"[0.16884927451610565, -0.27368104457855225, 0...."
2,đến,O,Sentence: 1,"[0.34477752447128296, 0.35272353887557983, 0.7..."
3,trường,O,Sentence: 1,"[0.2363833636045456, 0.08420327305793762, 0.50..."
4,của,O,Sentence: 1,"[0.12112768739461899, -0.495349645614624, 0.16..."
5,bé,O,Sentence: 1,"[0.2154824137687683, -0.18221479654312134, 0.5..."
6,gái,O,Sentence: 1,"[0.0816696509718895, 0.005837696138769388, 0.0..."
7,7,B-QUANTITY-AGE,Sentence: 1,"[0.08736534416675568, 0.0495893768966198, 0.02..."
8,tuổi,I-QUANTITY-AGE,Sentence: 1,"[-0.20265477895736694, -0.15798376500606537, 0..."
9,Giờ,B-DATETIME,Sentence: 2,"[0.33434125781059265, -0.016723310574889183, -..."


In [8]:
# Đếm số lượng nhãn
label_counts = df['Tag'].value_counts()

# In số lượng nhãn
print("Số lượng nhãn trong DataFrame:")
(label_counts)

Số lượng nhãn trong DataFrame:


Tag
O                         833546
B-PERSON                   15409
I-ORGANIZATION             11552
B-ORGANIZATION              9981
B-LOCATION-GPE              8831
                           ...  
I                             16
B-LOCATION-GPE HCM             6
A                              1
B-LOCATION-GPE-GEO             1
B-ORGANIZATION NN&PTNT         1
Name: count, Length: 90, dtype: int64

In [9]:
# pip install sklearn-crfsuite

# Feature for CRF

In [10]:
X = []
y = []

# Nhóm các từ theo câu dựa trên "Sentence #"
for sentence_id, group in df.groupby('Sentence #'):
    sentence_features = []
    sentence_tags = []

    for _, row in group.iterrows():
        # Tạo đặc trưng cho từ
        word_features = {
            'word': row['Word'],
        }
        # Thêm các giá trị từ vector làm đặc trưng riêng biệt
        for i, value in enumerate(row['Vector']):
            word_features[f'vector_{i}'] = value

        sentence_features.append(word_features)
        sentence_tags.append(row['Tag'])  # Bao gồm cả nhãn 'O'

    # Chỉ thêm các câu có ít nhất một nhãn
    if sentence_features:
        X.append(sentence_features)
        y.append(sentence_tags)

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


# Train model CRF

In [11]:
from sklearn_crfsuite import CRF
from sklearn.metrics import classification_report
import joblib

# Huấn luyện mô hình CRF với early stopping
crf = CRF(algorithm='lbfgs', max_iterations=100, all_possible_transitions=True, c1= 0.7146912130313807, c2= 0.020176090276753964)

best_f1 = 0.0
patience = 5
patience_counter = 0

for epoch in range(1, 100): 
    crf.fit(X_train, y_train)  # Huấn luyện mô hình

    # Dự đoán trên tập validation
    y_val_pred = crf.predict(X_val)
    
    # Chuyển đổi y_val và y_val_pred thành một dạng phù hợp
    y_val_flat = [item for sublist in y_val for item in sublist]
    y_val_pred_flat = [item for sublist in y_val_pred for item in sublist]

    # Tính điểm F1 cho tập validation
    report = classification_report(y_val_flat, y_val_pred_flat, output_dict=True)
    f1_score = report['weighted avg']['f1-score']
    
    print(f'Epoch: {epoch}, F1 Score: {f1_score:.4f}')

    # Lưu mô hình trong từng epoch
    joblib.dump(crf, f'/kaggle/working/best_model_crf.pkl')

    # Kiểm tra xem có cải thiện không
    if f1_score > best_f1:
        best_f1 = f1_score
        patience_counter = 0  # Reset counter
    else:
        patience_counter += 1
    
    # Kiểm tra điều kiện dừng
    if patience_counter >= patience:
        print("Early stopping activated.")
        break

print("Mô hình CRF đã được huấn luyện thành công.")


Epoch: 1, F1 Score: 0.9147
Epoch: 2, F1 Score: 0.9147
Epoch: 3, F1 Score: 0.9147
Epoch: 4, F1 Score: 0.9147
Epoch: 5, F1 Score: 0.9147
Epoch: 6, F1 Score: 0.9147
Early stopping activated.
Mô hình CRF đã được huấn luyện thành công.


In [12]:
# # Huấn luyện mô hình CRF với early stopping
# crf = CRF(algorithm='lbfgs', max_iterations=100, all_possible_transitions=True, verbose=True)

# best_f1 = 0.0
# patience = 5
# patience_counter = 0

# for epoch in range(1):  # Số lượng epoch tối đa
#     crf.fit(X_train, y_train)  # Huấn luyện mô hình

#     # Dự đoán trên tập validation
#     y_val_pred = crf.predict(X_val)
    
#     # Chuyển đổi y_val và y_val_pred thành một dạng phù hợp
#     y_val_flat = [item for sublist in y_val for item in sublist]
#     y_val_pred_flat = [item for sublist in y_val_pred for item in sublist]

#     # Tính điểm F1 cho tập validation
#     report = classification_report(y_val_flat, y_val_pred_flat, output_dict=True)
#     f1_score = report['weighted avg']['f1-score']
    
#     print(f'Epoch: {epoch+1}, F1 Score: {f1_score:.4f}')

#     # Kiểm tra xem có cải thiện không
#     if f1_score > best_f1:
#         best_f1 = f1_score
#         patience_counter = 0  # Reset counter
#     else:
#         patience_counter += 1
    
#     # Kiểm tra điều kiện dừng
#     if patience_counter >= patience:
#         print("Early stopping activated.")
#         break

# print("Mô hình CRF đã được huấn luyện thành công.")


In [13]:
# Chia dữ liệu thành tập huấn luyện, validation và kiểm tra
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # 20% cho tập kiểm tra

In [14]:
# Dự đoán trên tập kiểm tra
y_pred = crf.predict(X_test)

# Chuyển đổi y_test và y_pred thành một dạng phù hợp
y_test_flat = [item for sublist in y_test for item in sublist]
y_pred_flat = [item for sublist in y_pred for item in sublist]

# In báo cáo phân loại
print(classification_report(y_test_flat, y_pred_flat))

                       precision    recall  f1-score   support

                    B       0.00      0.00      0.00        12
            B-ADDRESS       0.19      0.27      0.22        15
           B-DATETIME       0.58      0.60      0.59       592
      B-DATETIME-DATE       0.60      0.57      0.58       352
 B-DATETIME-DATERANGE       0.36      0.22      0.27        82
  B-DATETIME-DURATION       0.54      0.73      0.62       301
       B-DATETIME-SET       0.25      0.07      0.11        14
      B-DATETIME-TIME       0.57      0.65      0.61        54
 B-DATETIME-TIMERANGE       0.64      0.53      0.58        87
              B-EMAIL       0.36      0.50      0.42         8
              B-EVENT       0.52      0.29      0.37       110
          B-EVENT-CUL       0.64      0.55      0.59        38
     B-EVENT-GAMESHOW       0.80      0.35      0.49        68
      B-EVENT-NATURAL       0.65      0.65      0.65        26
        B-EVENT-SPORT       0.83      0.73      0.78  

In [15]:
new_sentence = ["Minh","năm","nay","21","tuổi","xài","điện thoại","Apple"]


new_sentence_features = []
for word in new_sentence:
    word_features = {'word': word}
    
    word_vector = loaded_model.wv[word] 
    for i, value in enumerate(word_vector):
        word_features[f'vector_{i}'] = value
    
    new_sentence_features.append(word_features)

# Dự đoán nhãn cho câu mới bằng mô hình CRF
y_new_pred = crf.predict([new_sentence_features])

# In kết quả dự đoán nhãn
for word, label in zip(new_sentence, y_new_pred[0]):
    print(f"Từ: {word}, Nhãn dự đoán: {label}")


Từ: Minh, Nhãn dự đoán: B-PERSON
Từ: năm, Nhãn dự đoán: O
Từ: nay, Nhãn dự đoán: O
Từ: 21, Nhãn dự đoán: B-QUANTITY-AGE
Từ: tuổi, Nhãn dự đoán: I-QUANTITY-AGE
Từ: xài, Nhãn dự đoán: O
Từ: điện thoại, Nhãn dự đoán: O
Từ: Apple, Nhãn dự đoán: B-ORGANIZATION


# `Test`

In [16]:
import pickle
import joblib
from joblib import dump, load

# Load FastText model
try:
    with open('/kaggle/working/fasttext_model_f.pkl', 'rb') as f:
        model_fasttext = pickle.load(f)
except Exception as e:
    print(f"Error loading FastText model: {e}")
    
# Load CRF model    
try:
    with open('/kaggle/working/best_model_crf.pkl', 'rb') as c:
        model_crf = joblib.load(c)
except Exception as e:
    print(f"Error loading CRF model: {e}")

In [17]:
from nltk.tokenize import word_tokenize 

new_sentence ="Minh năm nay 21 tuổi đang học "

new_sentence = word_tokenize(new_sentence)

new_sentence_features = []
for word in new_sentence:
    word_features = {'word': word}
    
    word_vector = model_fasttext.wv[word] 
    for i, value in enumerate(word_vector):
        word_features[f'vector_{i}'] = value
    
    new_sentence_features.append(word_features)

# Dự đoán nhãn cho câu mới bằng mô hình CRF
y_new_pred = model_crf.predict([new_sentence_features])

# In kết quả dự đoán nhãn
for word, label in zip(new_sentence, y_new_pred[0]):
    print(f"Từ: {word}, Nhãn dự đoán: {label}")


Từ: Minh, Nhãn dự đoán: B-PERSON
Từ: năm, Nhãn dự đoán: O
Từ: nay, Nhãn dự đoán: O
Từ: 21, Nhãn dự đoán: B-QUANTITY-AGE
Từ: tuổi, Nhãn dự đoán: I-QUANTITY-AGE
Từ: đang, Nhãn dự đoán: O
Từ: học, Nhãn dự đoán: O
